: 

Circle Rasterization Method

In [9]:
from PIL import Image

def anger(xc, yc, rad):
    border_pixels = {}
    def add_border_pixel(x, y):
        border_pixels[(xc + x, yc + y)] = True
        border_pixels[(xc - x, yc + y)] = True
        border_pixels[(xc + x, yc - y)] = True
        border_pixels[(xc - x, yc - y)] = True
        border_pixels[(xc + y, yc + x)] = True
        border_pixels[(xc - y, yc + x)] = True
        border_pixels[(xc + y, yc - x)] = True
        border_pixels[(xc - y, yc - x)] = True
    x = 0
    y = rad
    d = 1 - rad
    add_border_pixel(x, y) 
    while x < y:
        if d < 0:
            d += 2 * x + 3
        else:
            d += 2 * (x - y) + 5
            y -= 1
        x += 1
        add_border_pixel(x, y)
    return border_pixels

#anti-ail helper functions
def blend(c1,c2, intensity):
    return tuple(int(c1[i] * intensity + c2[i] * (1 - intensity)) for i in range(3))
def intensity(x,y,rad):
    d = abs(x * x + y * y - rad * rad)
    return 1 - d / rad ** 2

#fill/reflect functions
def fill(pix, xl, xr, y, c):
        for x in range(xl, xr + 1):
            if (x, y) in border_pixels:
                pix[x, y] = blend(c,(0,0,0),0.4)
            elif (x,y) in bord2:
                pix[x, y] = blend(c,(0,0,0),0.8)
            elif (x,y) in bord3:
                pix[x, y] = blend(c,(0,0,0),0.95)
            else:
                pix[x, y] = c
def reflectPix(pix, xc, yc, x, y, c):
    fill(pix, xc - x, xc + x, yc + y, c)  
    fill(pix, xc - x, xc + x, yc - y, c) 
    fill(pix, xc - y, xc + y, yc + x, c) 
    fill(pix, xc - y, xc + y, yc - x, c)

#CircleFunction
def circleAlg(pix, xc, yc, c, rad):
    x = 0
    y = rad
    d = 1 - rad  #p0
    # Plot symmetric and fill
    reflectPix(pix, xc, yc, x, y, c)
    while x < y:
        if d < 0:#move right
            d += 2 * x + 3
        else:#move diagonally
            d += 2 * (x - y) + 5
            y -= 1
        x += 1
        reflectPix(pix, xc, yc, x, y, c)

bord2={}
bord3={}
border_pixels={}
def initiate():
    global bord2, bord3, border_pixels
    xsize=input("what x size")
    ysize=input("what y size")
    xsize,ysize=int(xsize),int(ysize)
    img = Image.new('RGB', (xsize, ysize))
    pix = img.load()
    xloc=input("what x location is the center")
    yloc=input("what x location is the center")
    xloc,yloc=int(xloc),int(yloc)
    radius=input("what radius do you want the circle")
    radius=int(radius)
    bord3=anger(xloc,yloc,radius-2)
    bord2=anger(xloc,yloc,radius-1)
    border_pixels = anger(xloc, yloc, radius)
    circleAlg(pix,xloc,yloc,(0,166,166),radius)
    large_size = (xsize * 3, ysize * 3)
    img_large = img.resize(large_size, Image.NEAREST)  # resize up to a larger size
    
    img_anti_aliased = img_large.resize((xsize, ysize), Image.LANCZOS)  # resize back down with LANCZOS filter
    
    # Show the anti-aliased image
    img_anti_aliased.show()


initiate()


ModuleNotFoundError: No module named 'PIL'

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Parameters for perspective projection
f = 500  # Focal length
d = 500  # Distance offset

# Read 3D points from face-vertices.data
vertices = []
with open('face-vertices.data', 'r') as file:
    for line in file:
        x, y, z = map(float, line.strip().split(','))
        vertices.append((x, y, z))

# Read indices from face-index.txt
triangles = []
with open('face-index.txt', 'r') as file:
    for line in file:
        indices = list(map(int, line.strip().split(',')))
        if len(indices) == 3:  # Ensure the line forms a triangle
            triangles.append((indices[0], indices[1], indices[2]))

# Convert 1-based indices to 0-based for Python
triangles = [(i1 - 1, i2 - 1, i3 - 1) for i1, i2, i3 in triangles]

# Function to project 3D points to 2D using perspective projection
def project_3d_to_2d(vertices, f, d):
    projected_points = []
    for x, y, z in vertices:
        x_2d = (x * f) / (z + d)
        y_2d = (y * f) / (z + d)
        projected_points.append((x_2d, y_2d))
    return projected_points

# Project vertices to 2D
projected_points = project_3d_to_2d(vertices, f, d)

# Display points mode
plt.figure(figsize=(10, 10))
plt.scatter(*zip(*projected_points), color='blue')
plt.title("Points Mode")
plt.gca().set_aspect('equal', adjustable='box')
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.show()

# Display wireframe mode
plt.figure(figsize=(10, 10))
for idx1, idx2, idx3 in triangles:
    x1, y1 = projected_points[idx1]
    x2, y2 = projected_points[idx2]
    x3, y3 = projected_points[idx3]
    plt.plot([x1, x2], [y1, y2], color='black')
    plt.plot([x2, x3], [y2, y3], color='black')
    plt.plot([x3, x1], [y3, y1], color='black')

plt.title("Wireframe Mode")
plt.gca().set_aspect('equal', adjustable='box')
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.show()
